In [1]:
# Uncomment the lines below to install if using google colab
# %%capture
# !pip install kinetics==1.4.1

In [ ]:
import kinetics
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import reciprocal, uniform, norm, lognorm
%config InlineBackend.figure_format ='retina'
print(kinetics.__version__)

## Model setup

In [ ]:
redam_fwd = kinetics.Ter_seq_redam(kcat="redam_kcat",
                                   kma="redam_km_nadph", kmb="redam_km_aldehyde", kmc="redam_km_nh2r",
                                   kia="redam_ki_nadph", kib="redam_ki_aldehyde",
                                   enz='redam', a="NADPH", b="Aldehyde", c="NH2R",
                                   substrates=["NADPH", "Aldehyde", "NH2R"],
                                   products=["NADP+", "Amine", "H2O"])

redam_fwd.parameter_distributions = {"redam_kcat" : (145,185),  
                                     "redam_km_nadph": (45,135), 
                                     "redam_km_aldehyde" : (84,112),  
                                     "redam_km_nh2r": (8600,15400), 
                                     "redam_ki_nadph": (150,450), 
                                     "redam_ki_aldehyde": (10,10000)}

redam_rev = kinetics.Ter_seq_redam(kcat="redam_kcat_rev",
                                   kma="redam_km_nadp", kmb="redam_km_amine", kmc="redam_km_h2o",
                                   kia="redam_ki_nadp", kib="redam_ki_amine",
                                   enz='redam', a="NADP+", b="Amine", c="H2O",
                                   substrates=["NADP+", "Amine", "H2O"],
                                   products=["NADPH", "Aldehyde", "NH2R"])
                                           
redam_rev.parameter_distributions = {"redam_kcat_rev" : (10,200),
                                     "redam_km_nadp" : (40,120),
                                     "redam_km_amine" : (10,10000), 
                                     "redam_km_h2o" : (10,10000), 
                                     "redam_ki_nadp" : (150,450), 
                                     "redam_ki_amine" : (10,10000)}

gdh = kinetics.Bi_ternary_complex(kcat='gdh_kcat',
                                  kma='gdh_km_nadp',
                                  kmb='gdh_km_gluc',
                                  kia='gdh_ki_nadp',
                                  a='NADP+', b='Glucose', enz='gdh',
                                  substrates=['NADP+', 'Glucose'],
                                  products=['NADPH', 'GDL'])

gdl_hydrolysis = kinetics.FirstOrderRate(k='gdl_hyd', a='GDL',
                                         substrates=['GDL'], products=['Gluconic Acid'])

gdh.parameter_distributions = {'gdh_kcat': (900,1084),  
                               'gdh_km_nadp' : (58, 78), 
                               'gdh_km_gluc' : (2100, 6300),  
                               'gdh_ki_nadp' : (38, 114),
                               'gdl_hyd' : (0.48,1.44)}

gdh.parameters = {'gdh_kcat': 992,  
                  'gdh_km_nadp' : 67, 
                  'gdh_km_gluc' : 3150,  
                  'gdh_ki_nadp' : 76,
                  'gdl_hyd' : 0.96}

# Set up the model
model = kinetics.Model()

# Add reactions
model.append(redam_fwd)
model.append(redam_rev)
model.append(gdh)
model.append(gdl_hydrolysis)

In [ ]:
# Set time
model.set_time(0, 24*60, 100)

# Set starting concentrations with error
model.species_distributions = {'redam' : (553, 773.87),   # +5%, -25%
                               'gdh' : (60, 84)} # +5%, -25%

# Command to prepare model for running.
model.setup_model()

In [ ]:
input_substrates_species = {}
input_substrates_list = []
substrates_list = list(model.species.keys())
immobilised = ['redam', 'gdh']

substrates_minus_enzymes = []

for substrate in substrates_list:
    if substrate not in immobilised:  
        input_substrates_list.append('input_' + str(substrate))
        substrates_minus_enzymes.append(substrate)

for substrate in input_substrates_list:
    input_substrates_species[substrate] = 0

input_substrates_species.update({"input_H2O" :55000000,
                                 'input_NADP+' : 500,
                                 'input_Glucose' : 20000,
                                 'input_Aldehyde' : 10000,
                                 'input_NH2R' : 15000})

In [ ]:
flow = kinetics.Flow(flow_rate='flow_rate', column_volume='column_volume',
                     input_substrates=input_substrates_list, substrates=substrates_minus_enzymes,
                     compartment_name='')


flow.parameters = ({'flow_rate' : 1,
                    'column_volume' : 1})

In [ ]:
model.append(flow)

model.species.update(input_substrates_species)

model.setup_model();

## Run model

In [ ]:
log = ["redam_ki_aldehyde", "redam_km_amine", "redam_km_h2o", "redam_ki_amine"]
samples = kinetics.sample_uniforms(model, num_samples=1000, log=log)
outputs = kinetics.run_all_models(model, samples, logging=True)
dataframes = kinetics.dataframes_all_runs(model, outputs)

In [ ]:
df95 = kinetics.dataframes_quartiles(model, outputs)

# Plot the confidence intervals
kinetics.plot_ci_intervals(['Aldehyde', 'Amine'], df95,
                           units=['μM', 'Time (mins)'],
                           colours=['darkred', 'darkgreen'],
                           alpha=0.1)
plt.show()

kinetics.plot_ci_intervals(['NADPH', 'NADP+'], df95,
                           units=['μM', 'Time (mins)'],
                           colours=['darkorange', 'lightblue'],
                           alpha=0.1)
plt.show()

kinetics.plot_ci_intervals(['GDL'], df95,
                           units=['μM', 'Time (mins)'],
                           colours=['darkblue'],
                           alpha=0.1)
plt.show()

## Metrics

In [ ]:
# Specify MW and initiate the metrics object
enzyme_mws = {'gdh' : 31500,
              'redam' : 31155}

species_mws = {'Aldehyde' : 134.178,
               'Amine' : 175.3}

metrics = kinetics.Metrics(model,
                           substrate='input_Aldehyde',
                           product='Amine',
                           reaction_volume=1/1000,
                           enzyme_mws=enzyme_mws,
                           species_mws=species_mws)

metrics.refresh_metrics(flow_rate='flow_rate')

print(metrics.biocatalyst_productivity())
print(metrics.space_time_yield())

## Sensitivity analysis

In [ ]:
log = ["redam_ki_aldehyde", "redam_km_amine", "redam_km_h2o", "redam_ki_amine"]
problem = kinetics.salib_problem(model, log=log)
sa_samples = kinetics.make_saltelli_samples(model, problem, 500, second_order=False)

In [ ]:
outputs = kinetics.run_all_models(model, sa_samples)

In [ ]:
timepoint = 60
substrate = 'Amine'
concs = kinetics.get_concentrations_at_timepoint(model, outputs, timepoint, substrate)
sobol = kinetics.analyse_sobal_sensitivity(problem, concs, second_order=False, num_resample=100, conf_level=0.95)

In [ ]:
new_df = kinetics.remove_st_less_than(sobol, column='ST', less_than=0.01)
kinetics.plot_sa_total_sensitivity(new_df)
plt.show()